In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url='http://api.scraperapi.com', #scrapeapi
            params={
                'api_key': 'fa9b49e2d89dc81fd7808deefca5c790',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v2.csv')
            line_count+=1

    

In [10]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("tmp_4_v7.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v2.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('tmp_5_v7.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [11]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/seshego-d/polokwane/limpopo/11150/109105061
{'name': '5 Bedroom House for Sale in Seshego D', 'price': 'R 790 000', 'province': 'limpopo', 'city': 'polokwane', 'address': 'Seshego D, Polokwane', 'Listing Number': '109105061', 'Type of Property': 'House', 'Listing Date': '11 September 2020', 'Erf Size': '600 m²', 'Pets Allowed': 'Yes', 'Bedrooms': '5', 'Bathrooms': '2', 'Kitchen': '1', 'Lounge': '1', 'Dining Room': '1', 'Family/TV Room': '1', 'Garage': '2', 'Security': '24 Hour Access', 'Education': ['0.15km', '0.26km', '0.44km', '0.45km', '1.13km', '1.13km', '1.23km', '1.25km', '1.29km', '1.33km', '1.39km', '1.40km', '1.57km', '1.63km', '1.68km', '1.75km', '1.81km', '1.92km', '1.97km', '2.09km', '2.10km', '2.14km', '2.18km', '2.24km', '2.26km', '2.43km', '2.71km', '2.96km', '2.97km', '3.50km']}
==>Property link:  /for-sale/umhlanga-ridge/umhlanga/kwazulu-natal/6383/110223486
{'name': '1 Bedroom Apartment / Flat for Sale in Umhlanga Ridge', 'price': 'R 1 090

{'name': '3 Bedroom House for Sale in Witbank Ext 8', 'price': 'R 1 080 000', 'province': 'mpumalanga', 'city': 'witbank', 'address': 'Witbank Ext 8, Witbank', 'Listing Number': '110034801', 'Type of Property': 'House', 'Description': 'Single Storey', 'Listing Date': '31 May 2021', 'Erf Size': '996 m²', 'Floor Size': '300 m²', 'Rates and Taxes': 'R 1 000', 'Pets Allowed': 'Yes', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchen': '1', 'Lounge': '1', 'Dining Room': '1', 'Family/TV Room': '1', 'Braai Room': '1', 'Garage': '1', 'Parking': '1', 'Garden': '1', 'Special Feature': 'Built in Braai', 'Flatlet': 'Yes', 'Education': ['1.48km', '1.63km', '1.81km', '1.86km', '1.91km', '1.92km', '1.97km', '2.20km', '2.69km', '2.74km', '2.82km', '2.89km', '2.90km', '3.01km', '3.04km', '3.05km', '3.43km', '3.46km', '3.68km', '3.78km', '3.98km', '4.07km', '4.17km', '4.24km', '4.26km', '4.37km'], 'Sports and Leisure': ['2.53km'], 'Shopping': ['4.27km', '4.44km'], 'Food and Entertainment': ['4.34km']}
==>Prope

KeyboardInterrupt: 